![QuantConnect Logo](https://cdn.quantconnect.com/web/i/icon.png)
<hr>

In [2]:
# QuantBook Analysis Tool 
# For more information see [https://www.quantconnect.com/docs/research/overview]
import matplotlib.pyplot as plt
import pandas as pd
import statsmodels
from statsmodels.tsa.stattools import adfuller

qb = QuantBook()
window = 365

In [3]:
    #Dickey-Fuller test stolen from https://www.hackdeploy.com/augmented-dickey-fuller-test-in-python/
    def ADF_Stationarity_Test(timeseries, significance_level=0.05, printResults = True):
        is_stationary = None
        
        adf_test = adfuller(timeseries, autolag='AIC')
        pValue = adf_test[1]
        
        if (pValue<significance_level):
            is_stationary = True
        else:
            is_stationary = False
        
        if printResults:
            dfResults = pd.Series(adf_test[0:4], index=['ADF Test Statistic','P-Value','# Lags Used','# Observations Used'])
            #Add Critical Values
            for key,value in adf_test[4].items():
                dfResults['Critical Value (%s)'%key] = value
            print('Augmented Dickey-Fuller Test Results:')
            print(dfResults)
            print('Is it stationary? '+str(is_stationary))

In [4]:
forex = qb.AddForex("USDCAD", Resolution.Daily, Market.Oanda).Symbol
forex_data = qb.History(forex, window-1)
print('USDCAD:')
forex_data['close'].plot()

In [12]:
oil = qb.AddCfd("WTICOUSD", Resolution.Daily, Market.Oanda).Symbol
oil_data = qb.History(oil, window)
plt.gca().invert_yaxis()
print('Crude Oil Inverted:')
oil_data['close'].plot()

In [7]:
print('Running Augmented Dickey-Fuller Test for Crude Oil...')
print()
ADF_Stationarity_Test(oil_data['close'].values)

In [8]:
# create a differenced series
def difference(series, interval=1):
    diff = []
    for i in range(interval, len(series)):
        value = series[i] - series[i - interval]
        diff.append(value)
    return diff


In [9]:
stationary_oil_data = pd.DataFrame(difference(oil_data['close'].values.tolist()))
print('Crued Oil Inverted:')
stationary_oil_data.plot()

In [11]:
print('Running Augmented Dickey-Fuller Test for Stationary Crude Oil...')
print()
ADF_Stationarity_Test(stationary_oil_data.values)